**NOTE** Please follow the instructions in README file before executing the cells in this Jupyter Notebook.

In [1]:
import pandas as pd
import numpy as np
import os
from Bio.PDB.MMCIFParser import MMCIFParser, MMCIF2Dict

In [2]:
RFAM_PATH =  # path to the Rfam covariance model
DATA_PATH = "../test/"
os.environ["DATA_PATH"] = DATA_PATH

In [3]:
import nucleoseeker.dataset_creator as dataset_creator
import nucleoseeker.dataset_download as dataset_download
import nucleoseeker.individual_structure_filter as individual_structure_filter
import nucleoseeker.columns as columns
import nucleoseeker.structure_comparison_filter as structure_comparison_filter
import nucleoseeker.metadata_filter as metadata_filter
import nucleoseeker.utils as utils

## Understanding the standalone use of each module

### How to use the `DatasetDownload` module?

This module is used to download data from the RCSB PDB database. It uses API calls to fetch the data. This provides a dataframe containing all the structures with their characteristics for the given criteria.

`dstart` and `dend` are used to specify the search results that are captured by the tool. If you want to analyse all available RNA data then `dstart=0` and `dend=10000` or `download_all=True` should be used. At later time, when number of RNA structures exceeds 10000, we would update this limit further.

The dataframe fetched can be accessed with the `df` attribute of the class.

Here is a simple usage example:

In [4]:
download = dataset_download.DatasetDownload(
    structure_determination_methodology = 'experimental',
    rcsb_entity_polymer_type = 'RNA',
    dstart = 0,
    dend = 100,
)

**Accessing the dataframe with `df` attribute**

In [5]:
rawdf = download.df

In [6]:
rawdf

,rcsb_id,exptl_method,release_date,polymer_entity_instance_count,polymer_entity_count_RNA,resolution,selected_polymer_entity_types,pdbx_keywords
0,124d,SOLUTION NMR,1993,2,1,NaN,Nucleic acid (only),DNA-RNA HYBRID
1,157d,X-RAY DIFFRACTION,1994,2,1,1.80,Nucleic acid (only),RNA
2,165d,X-RAY DIFFRACTION,1994,2,1,1.55,Nucleic acid (only),DNA-RNA HYBRID
3,176d,SOLUTION NMR,1994,2,1,NaN,Other,PEPTIDE NUCLEIC ACID/RNA
4,17ra,SOLUTION NMR,1999,1,1,NaN,Nucleic acid (only),RNA
...,...,...,...,...,...,...,...,...
95,1efo,X-RAY DIFFRACTION,2000,2,1,2.30,Nucleic acid (only),DNA-RNA HYBRID
96,1efs,SOLUTION NMR,2000,2,1,NaN,Nucleic acid (only),DNA-RNA HYBRID
97,1efw,X-RAY DIFFRACTION,2000,4,1,3.00,Protein/NA,LIGASE/RNA
98,1eg0,ELECTRON MICROSCOPY,2000,15,4,11.50,Protein/NA,RIBOSOME


### How to use the `MetadataFilter` module?

This module allows the application of structure level filters like experimental method, resolution and release year on a dataframe. It requires a dataframe to work, you can use the dataframe we obtained in the *previous step* or provide your own dataframe.

Make sure that the dataframe you provide has the same columns as given in `COLUMNS` module.

The filters of this module can be applied *individually* if you don't want to apply all of them together.

Here is a simple example:

In [7]:
struct_filter = metadata_filter.MetadataFilter(
    exptl_method = ['X-RAY DIFFRACTION'],
    resolution = 3.6,
    year_range = [1995, 2000],
    polymer_entity_instance_count=None,
    polymer_entity_count_RNA=None,
    selected_polymer_entity_types=None,
    pdbx_keywords=None
)

**Applying a single filter**

In [8]:
single_filter_df = struct_filter.apply_resolution_filter(rawdf)

In [9]:
single_filter_df

,rcsb_id,exptl_method,release_date,polymer_entity_instance_count,polymer_entity_count_RNA,resolution,selected_polymer_entity_types,pdbx_keywords
0,157d,X-RAY DIFFRACTION,1994,2,1,1.80,Nucleic acid (only),RNA
1,165d,X-RAY DIFFRACTION,1994,2,1,1.55,Nucleic acid (only),DNA-RNA HYBRID
2,1a34,X-RAY DIFFRACTION,1998,3,2,1.81,Protein/NA,Virus/RNA
3,1a9n,X-RAY DIFFRACTION,1998,6,1,2.38,Protein/NA,RNA BINDING PROTEIN/RNA
4,1aq3,X-RAY DIFFRACTION,1997,5,1,2.80,Protein/NA,Virus/RNA
5,1aq4,X-RAY DIFFRACTION,1997,5,1,3.00,Protein/NA,Virus/RNA
6,1asy,X-RAY DIFFRACTION,1995,4,1,2.90,Protein/NA,COMPLEX (AMINOACYL-TRNA SYNTHASE/TRNA)
7,1asz,X-RAY DIFFRACTION,1995,4,1,3.00,Protein/NA,COMPLEX (AMINOACYL-TRNA SYNTHASE/TRNA)
8,1av6,X-RAY DIFFRACTION,1998,2,1,2.80,Protein/NA,TRANSFERASE/RNA
9,1b23,X-RAY DIFFRACTION,1998,2,1,2.60,Protein/NA,GENE REGULATION/RNA


**Apply all filters together**

In [10]:
struct_filter_df = struct_filter.apply_filters(rawdf)

In [11]:
struct_filter_df

,rcsb_id,exptl_method,release_date,polymer_entity_instance_count,polymer_entity_count_RNA,resolution,selected_polymer_entity_types,pdbx_keywords
0,1asy,X-RAY DIFFRACTION,1995,4,1,2.9,Protein/NA,COMPLEX (AMINOACYL-TRNA SYNTHASE/TRNA)
1,1asz,X-RAY DIFFRACTION,1995,4,1,3.0,Protein/NA,COMPLEX (AMINOACYL-TRNA SYNTHASE/TRNA)
2,1csl,X-RAY DIFFRACTION,2000,2,2,1.6,Nucleic acid (only),RNA
3,1cwp,X-RAY DIFFRACTION,1995,6,2,3.2,Protein/NA,Virus/RNA
4,1ddl,X-RAY DIFFRACTION,2000,5,2,2.7,Protein/NA,Virus/RNA
5,1ddy,X-RAY DIFFRACTION,2000,4,1,3.0,Nucleic acid (only),RNA
6,1dk1,X-RAY DIFFRACTION,2000,2,1,2.8,Protein/NA,RIBOSOME
7,1dqf,X-RAY DIFFRACTION,2000,2,2,2.2,Nucleic acid (only),RNA
8,1dqh,X-RAY DIFFRACTION,2000,2,2,1.7,Nucleic acid (only),RNA
9,1duh,X-RAY DIFFRACTION,2000,1,1,2.7,Nucleic acid (only),RNA


### How to use the `IndividualStructureFilter` module?

Like all other modules of this tool, this also works as standalone and at the same time allows analysing PDB files which are not related to this tool.

Suppose you have a PDB file and you want to see if it satisfies some criteria then this module can be very handy. You only need to specify the PDB ID, and your criteria, this will download the PDB for you (if `auto_download=True`). It returns a list of tuples containing PDB ID, Chain ID, Sequence which satisfy your criteria.

Here you can see its usage:

**You will get a tuple if criteria is satisfied**

In [12]:
pdb_fil = individual_structure_filter.IndividualStructureFilter(
    pdb_id = '1efw',
    polymer_type = 'polyribonucleotide',
    sequence_length = 10,
    auto_download = True, # turn this to True if mmcif file is not available
    pdb_parser=MMCIF2Dict
)

In [13]:
data_list = pdb_fil.check_polymer_type()

In [15]:
data_list

[('1efw',
  'C',
  'GGAGCGGUAGUUCAGUCGGUUAGAAUACCUGCCUGUCACGCAGGGGGUCGCGGGUUCGAGUCCCGUCCGUUCC')]

**Empty list will be returned when criteria is not satisfied**

In [16]:
pdb_fil = individual_structure_filter.IndividualStructureFilter(
    pdb_id = '101m',
    polymer_type = 'polyribonucleotide',
    sequence_length = 10,
    auto_download = True, # turn this to True if mmcif file is not available
    pdb_parser=MMCIF2Dict
)

In [17]:
data_list = pdb_fil.check_polymer_type()

In [18]:
data_list

[]

### How to use the `StructureComparisonFilter` module?

In this module, we combine the `IndividualStructureFilter` module with an alignment software such as `ClustalOmega` or `Emboss` such that it can process multiple PDB files and also filter them on the basis of sequence identity. 

For this section you need to have `ClustalOmega` installed otherwise it will fail. (See README)

You specify the criteria and the threshold of sequence identity. It calculates pairwise sequence identity between each structure and for structures with an identity percentage above the threshold (say `sequence_identity=50.00`) it chooses the one with higher resolution.

We recommend (unless you want to experiment) using `ClustalOmega` for your analysis as it is much efficient than `'Emboss`.

**This can be applied on a single PDB file**

In [19]:
poly_fil = structure_comparison_filter.StructureComparisonFilter(
    polymer_type = 'polyribonucleotide',
    sequence_length = 10,
    sequence_identity = 50.00,
    auto_download = True,
    alignment_tool = 'clustal'
)

In [20]:
single_pdb_data_list = poly_fil.apply_filters_on_pdb_id('1cgm')

In [21]:
single_pdb_data_list

[]

**It also works on a list of PDB files**

In [22]:
multiple_pdbs_data_list = poly_fil.apply_filters_on_list(['1ehz', '1aud'])

In [23]:
multiple_pdbs_data_list

[('1ehz',
  'A',
  'GCGGAUUUAGCUCAGUUGGGAGAGCGCCAGACUGAAGAUCUGGAGGUCCUGUGUUCGAUCCACAGAAUUCGCACCA'),
 ('1aud', 'B', 'GGCAGAGUCCUUCGGGACAUUGCACCUGCC')]

**Finally, if you provide this a dataframe with other filters applied and a data_list (obtained at *previous step*), this will apply the sequence identity criteria on the dataframe and return the final dataframe**

In [24]:
data_list = poly_fil.apply_filters_on_list(struct_filter_df['rcsb_id'].to_list())
final_df = poly_fil.apply_filter_on_df(struct_filter_df, data_list)


**Here is the final dataframe with all the filters applied**

In [25]:
final_df

,rcsb_id,exptl_method,release_date,polymer_entity_instance_count,polymer_entity_count_RNA,resolution,selected_polymer_entity_types,pdbx_keywords,chain_id,sequence
2,1csl,X-RAY DIFFRACTION,2000,2,2,1.6,Nucleic acid (only),RNA,A,AACGGGCGCAGAA
6,1dqh,X-RAY DIFFRACTION,2000,2,2,1.7,Nucleic acid (only),RNA,B,CAGGGUCGGC
8,1dul,X-RAY DIFFRACTION,2000,2,1,1.8,Protein/NA,SIGNALING PROTEIN/RNA,B,GGCUCUGUUUACCAGGUCAGGUCCGAAAGGAAGCAGCCAAGGCAGAGCC
12,1efo,X-RAY DIFFRACTION,2000,2,1,2.3,Nucleic acid (only),DNA-RNA HYBRID,B,GAAGAGAGAG


**NOTE**: In the module `DatasetCreator`, we combine all the filters and search for the families of these structures using `cmscan` from `Infernal`.